In [1]:
# read txt document of entry points
# also iterate over all tif files (adult 3D mask images) in the same path and perform analysis 
# all analysis except column analysis 

In [2]:
#  input and save paths
data_path = '/home/tli_project/Desktop/Data/T4/neurons/SubtypeB/'
save_path = '/home/tli_project/Desktop/Data/T4/neurons/SubtypeB/output/'
subtype = 'B'
start_age = 37

In [3]:
import os
import utils.analysis as analysis
import utils.datautils as datautils
import matplotlib.pyplot as plt
import tifffile as tif
import numpy as np
import re
from tqdm import tqdm
import pandas as pd
import math
# from skimage.measure import regionprops
from read_roi import read_roi_zip as col_zip
# from scipy.spatial import ConvexHull, convex_hull_plot_2d
from scipy import ndimage, spatial

In [4]:
# getting list of folders in data_path, where each folder has files for one neuron
N_folders = [name for name in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, name))]
N_folders = [f for f in N_folders if '_P' in f]
N_folders = sorted(N_folders)
N_folders

['211011_P37_N1', '211107_P36_N1']

In [5]:
# reading erntry_points file and converting it to dictionary of neu:[z,y,x]
# EP_file = '/home/tli_project/Desktop/Data/T4/neurons/TLI_stage1_entry_points'
# with open(EP_file) as f:
#     entry_points = f.readlines()
# entry_points = [l for l in entry_points if ': ' in l]
# entry_points = [l.rstrip('\n').split(':') for l in entry_points]
# entry_points = {l[0]: l[1].split(',') for l in entry_points}
# entry_points = {neu:[int(x) for x in val] for neu,val in entry_points.items()}

In [10]:
def get_cols_labeled_neuron(n_file):
    # entry_point = entry_points[n_file][1:] # remove the first T element
    start_im = re.search('P(.*)_N', n_file)
    start_im = float(start_im.group(1))
    start_ana = int((start_age - start_im)/0.25)
    files_list = datautils.get_file_names(data_path+n_file, group_by='', 
                                        order=True, nested_files=False, 
                                        criteria='')
    neu_path = [f for f in files_list if 'clahe' in f.lower()][0]
    neuron = tif.imread(neu_path)
    mask_path = [f for f in files_list if 'mask' in f.lower()][0]
    mask = tif.imread(mask_path)
    neuron[mask==0] = 0
    neuron[neuron != 0] = 1
    # in case you want to crop the image to certain timeponit-start 
    # neuron = neuron[start_ana:] #remove timepoints before start_ana from the 4D image array
    
    # openning the cols_zip file
    cols_path = [f for f in files_list if 'colrois' in f.lower()][0]
    Cols = col_zip(cols_path)

    # get columns centroids (only three are needed but the code is simpler this way)
    cols_centroids = {}
    for col, val in Cols.items():
        if val['type'] == 'oval':
                x0 = val['left']+int(val['width']/2); a = int(val['width']/2)  # x center, half width                                       
                y0 = val['top']+int(val['height']/2); b = int(val['height']/2)  # y center, half height  
                cols_centroids[col] = (y0,x0,a,b) 
    
    #[N,2] array of centroids of all the columns
    cols_centroids_arr = np.array([(i[0],i[1]) for i in cols_centroids.values()])
    cols_names = list(cols_centroids.keys())
    cols_px_vals = {col:i+1 for i, col in enumerate(cols_centroids.keys())}

    # get the mask of each col based on distances from centroids 
    neu_PC = np.argwhere(np.ones_like(neuron[0,0])) #getting a point cloud of all the possible px in 2D [Y,X] array
    # cols_masks = {col:np.zeros_like(neuron[0,0]) for col in cols_centroids.keys()}
    cols_masks = np.zeros_like(neuron[0,0])
    for px in tqdm(neu_PC):
        dist, ind = spatial.KDTree(cols_centroids_arr).query(px)
        col = cols_names[ind]
        cols_masks[px[0],px[1]] = cols_px_vals[col]

    # col_labeled_neuron = np.zeros_like(neuron)
    
    mask_4D = np.broadcast_to(cols_masks, neuron.shape)
    col_labeled_neuron = neuron * mask_4D

    # for col, val in cols_px_vals.items():
    #     Roi_3D = np.broadcast_to(cols_masks[col], neuron.shape)
    #     col_mask = Roi_3D * neuron
    #     col_mask = col_mask * val
        # col_labeled_neuron = col_labeled_neuron + col_mask
    
    # saving the result labeled image
    col_labeled_neuron = col_labeled_neuron.astype('uint8')
    name = save_path+n_file+'_cols_labeled.tif'
    tif.imwrite(name, col_labeled_neuron, imagej=True, resolution=(1./0.076, 1./0.076),
                metadata={'spacing': 0.4, 'unit': 'um', 'finterval': 1/10, 'axes':'TZYX'})
    
    # option for saving the mask
    name = save_path+n_file+'_cols_mask.tif'
    tif.imwrite(name, cols_masks, imagej=True, resolution=(1./0.076, 1./0.076),
                metadata={'spacing': 0.4, 'unit': 'um', 'finterval': 1/10, 'axes':'YX'})

    return col_labeled_neuron, cols_masks

In [11]:
for n_file in tqdm(N_folders):
    col_labeled_neuron, cols_masks = get_cols_labeled_neuron(n_file)

  0%|          | 0/2 [00:00<?, ?it/s]

first 5 files
['/home/tli_project/Desktop/Data/T4/neurons/SubtypeB/211011_P37_N1/test_line.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeB/211011_P37_N1/original_cols_3.png', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeB/211011_P37_N1/original_cols_2.png', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeB/211011_P37_N1/original_cols_10.png', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeB/211011_P37_N1/mywarpedgrid_3.png']


 50%|█████     | 1/2 [00:09<00:09,  9.98s/it]

first 5 files
['/home/tli_project/Desktop/Data/T4/neurons/SubtypeB/211107_P36_N1/211107_P36_N1_Z30.70_clahe.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeB/211107_P36_N1/211107_P36_N1_Z30.70_5D.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeB/211107_P36_N1/211107_P36_N1_R265_mask1.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeB/211107_P36_N1/211107_P36_N1_R265_Z30.70_5D.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeB/211107_P36_N1/211107_P36_N1_R265_ColRois.zip']


100%|██████████| 2/2 [00:19<00:00,  9.56s/it]
